# Net Position Analysis: From Net-to-Robot to Robot

This notebook analyzes the relationship between:
1. **Net position from robot's perspective** (sonar-detected distance and angle)
2. **Robot position relative to the net** (navigation-based positioning)

The analysis transforms between these two coordinate systems using CSV data from net detection results and navigation data.

## 1. Import Libraries and Utils

Import necessary libraries and our custom net position analysis utilities.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path

# Import our custom net position analysis utilities
from utils.net_position_analysis import (
    load_net_analysis_results,
    load_navigation_data,
    calculate_robot_to_net_position,
    analyze_net_position_consistency,
    create_net_position_visualization,
    run_net_position_analysis
)

# Import configuration
from utils.sonar_config import EXPORTS_DIR_DEFAULT

print("Libraries and utilities imported successfully!")

Libraries and utilities imported successfully!


## 2. Load Net Results from CSV

Load the net analysis results that contain sonar-detected distances and angles to the net from the robot's perspective.

In [2]:
# Configuration - set your target bag name here
TARGET_BAG = "2024-08-20_17-02-00"  # Updated to match FFT data file
EXPORTS_FOLDER = Path(EXPORTS_DIR_DEFAULT)

print(f"Target Bag: {TARGET_BAG}")
print(f"Exports Folder: {EXPORTS_FOLDER}")

# NOTE: If you need to export all data for this specific bag, you can use:
# python3 scripts/solaqua_export.py --bag-stem 2024-08-20_17-02-00 --all
# This will export CSV, video, frames, camera info, and NPZ for just this bag

# Load net analysis results (sonar detections from robot perspective)
net_df = None
try:
    net_df = load_net_analysis_results(TARGET_BAG, EXPORTS_FOLDER)
    print(f"✓ Loaded {len(net_df)} net analysis frames")
    print(f"Columns: {list(net_df.columns)}")
    print(f"Detection success rate: {net_df['detection_success'].mean()*100:.1f}%")
except ModuleNotFoundError as e:
    print(f"✗ Module not found for automatic analysis creation: {e}")
    print("\nTo proceed, you have these options:")
    print(f"1. Run image analysis manually using notebook 06")
    print(f"2. Use an existing analysis CSV file")
    print(f"3. Run: python scripts/solaqua_export.py --bag-stem {TARGET_BAG} --csv --npz")
    print("   Then run distance analysis in notebook 06 with save_outputs=True")
    net_df = None
except FileNotFoundError as e:
    print(f"✗ Error loading net results: {e}")
    net_df = None
except Exception as e:
    print(f"✗ Unexpected error: {e}")
    net_df = None

if net_df is None:
    print(f"\nCannot proceed without analysis results for {TARGET_BAG}")
    print("Please ensure you have either:")
    print("  - Existing analysis CSV files, or") 
    print("  - NPZ files and the image analysis module")

Target Bag: 2024-08-20_17-02-00
Exports Folder: /Volumes/LaCie/SOLAQUA/exports
Loaded analysis file: 2024-08-20_17-02-00_data_cones_analysis_results.csv (944 frames)
✓ Loaded 944 net analysis frames
Columns: ['frame_index', 'timestamp', 'distance_pixels', 'angle_degrees', 'distance_meters', 'detection_success', 'tracking_status', 'area', 'aspect_ratio', 'solidity', 'extent', 'ellipse_elongation', 'straightness', 'rect', 'centroid_x', 'centroid_y']
Detection success rate: 100.0%
Loaded analysis file: 2024-08-20_17-02-00_data_cones_analysis_results.csv (944 frames)
✓ Loaded 944 net analysis frames
Columns: ['frame_index', 'timestamp', 'distance_pixels', 'angle_degrees', 'distance_meters', 'detection_success', 'tracking_status', 'area', 'aspect_ratio', 'solidity', 'extent', 'ellipse_elongation', 'straightness', 'rect', 'centroid_x', 'centroid_y']
Detection success rate: 100.0%


## 2a. Load Sonar Analysis Results and Navigation Data

Load the CSV files containing sonar distance/angle detections and DVL navigation data for comparison.

In [3]:
# Load sonar distance analysis results - with automatic creation if needed
print("Loading sonar analysis results...")

try:
    df_sonar = load_net_analysis_results(TARGET_BAG, EXPORTS_FOLDER)
    print(f"✓ Loaded sonar analysis: {len(df_sonar)} frames")
    print(f"Columns: {list(df_sonar.columns)}")
    print(f"Detection success rate: {df_sonar['detection_success'].mean()*100:.1f}%")
except FileNotFoundError as e:
    print(f"✗ Error: {e}")
    df_sonar = None
except Exception as e:
    print(f"✗ Unexpected error loading sonar analysis: {e}")
    df_sonar = None

if df_sonar is None:
    print("\nCannot proceed without sonar analysis results.")
    print("Please ensure you have the required NPZ file or existing analysis results.")

Loading sonar analysis results...
Loaded analysis file: 2024-08-20_17-02-00_data_cones_analysis_results.csv (944 frames)
✓ Loaded sonar analysis: 944 frames
Columns: ['frame_index', 'timestamp', 'distance_pixels', 'angle_degrees', 'distance_meters', 'detection_success', 'tracking_status', 'area', 'aspect_ratio', 'solidity', 'extent', 'ellipse_elongation', 'straightness', 'rect', 'centroid_x', 'centroid_y']
Detection success rate: 100.0%


In [4]:
# Load navigation data (DVL) - EXACT same method as video generation
print("Loading DVL data using video generation method...")

# Use the same file pattern as the video generation code
nav_file = EXPORTS_FOLDER / "by_bag" / f"navigation_plane_approximation__{TARGET_BAG}_data.csv"

if nav_file.exists():
    print(f"✓ Found navigation file: {nav_file.name}")
    df_nav = pd.read_csv(nav_file)
    
    # Apply same transformations as video generation code
    df_nav["timestamp"] = pd.to_datetime(df_nav["ts_utc"])
    df_nav = df_nav.sort_values("timestamp")
    
    print(f"✓ Loaded navigation data: {len(df_nav)} records")
    print(f"Columns: {list(df_nav.columns)}")
    
    # Check for the key columns that video generation uses
    key_columns = [c for c in ["NetDistance", "NetPitch", "timestamp"] if c in df_nav.columns]
    print(f"Key DVL columns available: {key_columns}")
    
    # Display sample data focusing on the columns used by video generation
    if 'NetDistance' in df_nav.columns:
        valid_distances = df_nav['NetDistance'].dropna()
        print(f"NetDistance: {len(valid_distances)} valid measurements")
        print(f"  Range: {valid_distances.min():.3f} to {valid_distances.max():.3f} m")
        print(f"  Mean: {valid_distances.mean():.3f} ± {valid_distances.std():.3f} m")
    
    if 'NetPitch' in df_nav.columns:
        valid_pitch = df_nav['NetPitch'].dropna()
        print(f"NetPitch: {len(valid_pitch)} valid measurements")
        print(f"  Range: {np.degrees(valid_pitch.min()):.1f}° to {np.degrees(valid_pitch.max()):.1f}°")
        print(f"  Mean: {np.degrees(valid_pitch.mean()):.1f}° ± {np.degrees(valid_pitch.std()):.1f}°")
    
    print(f"\nSample navigation data (first 3 rows):")
    display_cols = ['timestamp', 'NetDistance', 'NetPitch']
    available_cols = [c for c in display_cols if c in df_nav.columns]
    print(df_nav[available_cols].head(3))
    
else:
    print(f"✗ Navigation file not found: {nav_file}")
    
    # Try the alternative loading method as fallback
    print("Trying alternative loading method...")
    try:
        import utils.net_distance_analysis as sda
        BY_BAG_FOLDER = EXPORTS_FOLDER / "by_bag"
        raw_data, _ = sda.load_all_distance_data_for_bag(TARGET_BAG, BY_BAG_FOLDER)
        
        if raw_data and 'navigation' in raw_data and raw_data['navigation'] is not None:
            df_nav = raw_data['navigation'].copy()
            df_nav['timestamp'] = pd.to_datetime(df_nav['timestamp'], errors='coerce')
            print(f"✓ Loaded via fallback method: {len(df_nav)} records")
        else:
            df_nav = None
            print("✗ No navigation data found via fallback method")
    except Exception as e:
        print(f"✗ Fallback loading failed: {e}")
        df_nav = None

Loading DVL data using video generation method...
✓ Found navigation file: navigation_plane_approximation__2024-08-20_17-02-00_data.csv
✓ Loaded navigation data: 497 records
Columns: ['t', 't_header', 't_bag', 't_src', 'bag', 'bag_file', 'topic', 'NormalDVL', 'Altitude', 'NetDistance', 'NetHeading', 'NetPitch', 'NetLock', 'NetVelocity_u', 'NetVelocity_v', 'NetVelocity_w', '__msgtype__', 't0', 't_rel', 'ts_utc', 'ts_oslo', 'timestamp']
Key DVL columns available: ['NetDistance', 'NetPitch', 'timestamp']
NetDistance: 497 valid measurements
  Range: 0.430 to 3.340 m
  Mean: 1.121 ± 0.305 m
NetPitch: 497 valid measurements
  Range: -60.5° to 85.3°
  Mean: -1.8° ± 13.0°

Sample navigation data (first 3 rows):
                            timestamp  NetDistance  NetPitch
0 2024-08-20 15:02:05.407151699+00:00         0.72  -0.07037
1 2024-08-20 15:02:05.502894639+00:00         0.76  -0.09843
2 2024-08-20 15:02:05.603660345+00:00         0.77  -0.06585


In [5]:
# Verify both datasets are loaded before comparison
if df_sonar is not None and df_nav is not None:
    print("✓ Both datasets loaded successfully")
    print(f"Sonar: {len(df_sonar)} frames, {df_sonar['detection_success'].sum()} successful")
    print(f"Navigation: {len(df_nav)} records")
    
    # Set net_df to df_sonar for compatibility with existing code
    net_df = df_sonar.copy()
    print(f"✓ Set net_df for compatibility")
else:
    print("✗ Cannot proceed - missing sonar or navigation data")
    net_df = None

✓ Both datasets loaded successfully
Sonar: 944 frames, 944 successful
Navigation: 497 records
✓ Set net_df for compatibility


In [6]:
# Calculate robot-to-net positions (transform sonar detections to world coordinates)
if net_df is not None and df_nav is not None:
    print("Calculating robot-to-net positions...")
    combined_df = calculate_robot_to_net_position(net_df, df_nav)

    print(f"✓ Generated {len(combined_df)} position calculations")
    print(f"Successful detections: {combined_df['detection_success'].sum()}")

    # Show sample of combined data
    print("\nSample combined data:")
    print(combined_df.head())

    # Summary statistics
    print("\nPosition Summary:")
    print(f"Mean sonar distance: {combined_df['sonar_distance_m'].mean():.2f} m")
    print(f"Mean sonar angle: {combined_df['sonar_angle_deg'].mean():.2f}°")
    print(f"Net position range - X: {combined_df['net_x_world'].min():.2f} to {combined_df['net_x_world'].max():.2f} m")
    print(f"Net position range - Y: {combined_df['net_y_world'].min():.2f} to {combined_df['net_y_world'].max():.2f} m")
else:
    print("✗ Cannot proceed - missing data")
    combined_df = None

Calculating robot-to-net positions...
Timestamp range - Sonar: 2024-08-20 15:02:03.304299593 to 2024-08-20 15:03:03.646958113
Timestamp range - Nav:   2024-08-20 15:02:05.407151699 to 2024-08-20 15:03:02.794224739
Merged 944 records from 944 sonar and 497 nav records
✓ Generated 944 position calculations
Successful detections: 944

Sample combined data:
                      timestamp  frame_index  detection_success  \
0 2024-08-20 15:02:03.304299593            1               True   
1 2024-08-20 15:02:03.392066002            2               True   
2 2024-08-20 15:02:03.444470167            3               True   
3 2024-08-20 15:02:03.490314007            4               True   
4 2024-08-20 15:02:03.596029997            5               True   

   sonar_distance_m  sonar_angle_deg  robot_x  robot_y  robot_heading  \
0          0.600799       170.797546      0.0      0.0       -1.15479   
1          0.596682       171.085663      0.0      0.0       -1.15479   
2          0.595283   

## 4. Analyze Position Consistency

Analyze the consistency of net position estimates and calculate statistics.

In [7]:
# Analyze consistency of net position estimates
consistency_stats = analyze_net_position_consistency(combined_df)

print("=== POSITION CONSISTENCY ANALYSIS ===")
for key, value in consistency_stats.items():
    if isinstance(value, float):
        print(f"{key}: {value:.3f}")
    else:
        print(f"{key}: {value}")

# Filter to successful detections for detailed analysis
success_df = combined_df[combined_df['detection_success']].copy()
print(f"\nDetailed analysis of {len(success_df)} successful detections:")

if len(success_df) > 0:
    # Calculate position variability
    position_variability = np.sqrt(
        (success_df['net_x_world'] - consistency_stats['net_centroid_x'])**2 +
        (success_df['net_y_world'] - consistency_stats['net_centroid_y'])**2
    )

    print(f"95th percentile position error: {np.percentile(position_variability, 95):.3f} m")
    print(f"Maximum position error: {position_variability.max():.3f} m")

=== POSITION CONSISTENCY ANALYSIS ===
total_frames: 944
successful_detections: 944
detection_rate: 100.000
net_centroid_x: -1.141
net_centroid_y: 0.362
position_std_m: 0.746
position_95th_percentile_m: 2.829
sonar_distance_mean: 1.300
sonar_distance_std: 0.798
sonar_angle_std: 15.002

Detailed analysis of 944 successful detections:
95th percentile position error: 2.829 m
Maximum position error: 3.892 m


In [8]:
# Load and process relative FFT data
print("=== LOADING RELATIVE FFT DATA ===")

from utils.relative_fft_analysis import (
    load_relative_fft_data,
    analyze_fft_data_quality,
    convert_fft_to_xy_coordinates,
    synchronize_fft_with_other_systems,
    create_three_system_comparison_plot
)

# Configuration for FFT data
FFT_CSV_PATH = Path("/Volumes/LaCie/SOLAQUA/relative_fft_pose/2024-08-20_17-02-00_relative_pose_fft.csv")

print(f"FFT CSV Path: {FFT_CSV_PATH}")
print(f"File exists: {FFT_CSV_PATH.exists()}")

# Load FFT data if file exists
df_fft = None
if FFT_CSV_PATH.exists():
    try:
        df_fft = load_relative_fft_data(FFT_CSV_PATH)
        df_fft = convert_fft_to_xy_coordinates(df_fft)
        
        # Analyze FFT data quality
        fft_stats = analyze_fft_data_quality(df_fft)
        
        print(f"✓ Loaded FFT data: {len(df_fft)} records")
        print(f"  Time range: {fft_stats['time_range_seconds']:.1f} seconds")
        print(f"  Distance range: {fft_stats['distance_min_m']:.3f} - {fft_stats['distance_max_m']:.3f} m")
        print(f"  Pitch range: {fft_stats['pitch_min_deg']:.1f}° - {fft_stats['pitch_max_deg']:.1f}°")
        
    except Exception as e:
        print(f"✗ Error loading FFT data: {e}")
        df_fft = None
else:
    print(f"✗ FFT data file not found: {FFT_CSV_PATH}")
    print("  Continuing with two-system comparison only")

=== LOADING RELATIVE FFT DATA ===
FFT CSV Path: /Volumes/LaCie/SOLAQUA/relative_fft_pose/2024-08-20_17-02-00_relative_pose_fft.csv
File exists: True
✓ Loaded 410 relative FFT pose records
✓ Loaded FFT data: 410 records
  Time range: 56.8 seconds
  Distance range: -1.833 - 1.596 m
  Pitch range: -82.7° - 88.1°


## 7. Three-System Synchronized Comparison

Synchronize and compare measurements from all three systems: Sonar, DVL, and Relative FFT.

In [ ]:
# COMPREHENSIVE THREE-SYSTEM NET RELATIVE POSITION ANALYSIS
print("=== THREE-SYSTEM NET RELATIVE POSITION ANALYSIS ===")

if df_sonar is not None and df_nav is not None and df_fft is not None:
    # Import the new net relative utilities
    from utils.net_relative_utils import run_complete_three_system_analysis
    
    print("Running comprehensive three-system analysis with net relative position calculations...")
    
    # Run the complete analysis
    sync_df, stats, fig_distance, fig_pitch, fig_x_position, fig_y_position = run_complete_three_system_analysis(
        df_sonar=df_sonar,
        df_nav=df_nav, 
        df_fft=df_fft,
        target_bag=TARGET_BAG,
        exports_folder=EXPORTS_FOLDER
    )
    
    # Display all four separate visualizations
    print("\nDisplaying distance comparison:")
    fig_distance.show()
    
    print("\nDisplaying pitch comparison:")
    fig_pitch.show()
    
    print("\nDisplaying X position comparison:")
    fig_x_position.show()
    
    print("\nDisplaying Y position comparison:")
    fig_y_position.show()
    
    # Print analysis summary
    if stats:
        print(f"\n=== ANALYSIS SUMMARY ===")
        print(f"Total synchronized points: {stats.get('total_sync_points', 0)}")
        print(f"Valid distance comparisons: {stats.get('valid_distance_points', 0)}")
        
        if 'fft_sonar_distance_corr' in stats:
            print(f"\nDistance correlations:")
            print(f"  FFT ↔ Sonar: {stats['fft_sonar_distance_corr']:.3f}")
            print(f"  FFT ↔ DVL: {stats['fft_nav_distance_corr']:.3f}")
            print(f"  Sonar ↔ DVL: {stats['sonar_nav_distance_corr']:.3f}")
    
    print(f"\nComplete three-system net relative position analysis finished")
    print(f"Color coding: Red=FFT, Blue=Sonar, Green=DVL")
    print(f"Note: Sonar angles corrected by -180° for coordinate system alignment")
    
else:
    print("Missing one or more datasets - cannot perform three-system comparison")
    print("Available data:")
    print(f"  Sonar: {'✓' if df_sonar is not None else '✗'}")
    print(f"  Navigation: {'✓' if df_nav is not None else '✗'}")
    print(f"  FFT: {'✓' if df_fft is not None else '✗'}")

=== THREE-SYSTEM NET RELATIVE POSITION ANALYSIS ===
Running comprehensive three-system analysis with net relative position calculations...
Step 1: Calculating net relative positions
FFT distances appear to be in cm (max: 183.3), converting to meters
Final FFT distance range: -1.833 to 1.596 m
FFT pitch range: -82.7° to 88.1°
FFT X range: -1.821 to 1.573 m
FFT Y range: -0.596 to 0.441 m
Step 2: Synchronizing all three systems
Synchronized 1847 time points
Available columns in sync_df: ['sync_timestamp', 'fft_distance_m', 'fft_pitch_deg', 'fft_x_m', 'fft_y_m', 'fft_heading', 'sonar_distance_m', 'sonar_pitch_deg', 'sonar_x_m', 'sonar_y_m', 'sonar_detection_success', 'nav_distance_m', 'nav_pitch_deg', 'nav_x_m', 'nav_y_m']
Available distance columns: ['fft_distance_m', 'sonar_distance_m', 'nav_distance_m']
Found distance columns: FFT=fft_distance_m, Sonar=sonar_distance_m, Nav=nav_distance_m
Step 3: Creating visualizations
Position data summary:
  FFT: 1765 points, X: 0.82±0.38, Y: 0.04±0.


Displaying pitch comparison:



Displaying X position comparison:



Displaying Y position comparison:



=== ANALYSIS SUMMARY ===
Total synchronized points: 1847
Valid distance comparisons: 1735

Distance correlations:
  FFT ↔ Sonar: -0.161
  FFT ↔ DVL: 0.040
  Sonar ↔ DVL: 0.722

✅ Complete three-system net relative position analysis finished
Color coding: Red=FFT, Blue=Sonar, Green=DVL
Note: Sonar angles corrected by -180° for coordinate system alignment


## Data Transformation and Synchronization Pipeline

This section explains how data from the three systems (Sonar, DVL, FFT) is processed to make them comparable:

### 1. **Raw Data Sources & Formats**

#### Sonar Data (Robot's perspective of net):
- **Source**: `load_net_analysis_results()` from sonar image analysis
- **Raw format**: 
  - `sonar_distance_m`: Distance from robot to net (meters)
  - `sonar_angle_deg`: Angle to net from robot's heading (degrees)
  - `timestamp`: Time of detection
- **Coordinate system**: Robot-centric polar coordinates

#### DVL Navigation Data (Robot position relative to net):
- **Source**: `navigation_plane_approximation_{TARGET_BAG}_data.csv`
- **Raw format**:
  - `NetDistance`: Robot's distance from net plane (meters)
  - `NetPitch`: Robot's pitch angle relative to net (radians)
  - `ts_utc`: UTC timestamp
- **Coordinate system**: Net-centric Cartesian coordinates

#### FFT Relative Data (High-precision net detection):
- **Source**: `{TARGET_BAG}_relative_pose_fft.csv`
- **Raw format**:
  - `distance_m`: FFT-calculated distance to net (meters)
  - `pitch_deg`: FFT-calculated pitch angle (degrees)
  - `timestamp`: Time of measurement
- **Coordinate system**: FFT-derived polar coordinates

### 2. **Coordinate System Transformations**

#### A. Sonar → World Coordinates:
```python
# Transform sonar detections to world coordinates
combined_df = calculate_robot_to_net_position(net_df, df_nav)
# Creates: net_x_world, net_y_world from sonar_distance_m, sonar_angle_deg
```

#### B. DVL → Standard Units:
```python
# Convert DVL data to standard format
df_nav["timestamp"] = pd.to_datetime(df_nav["ts_utc"])
# NetPitch already in radians, NetDistance in meters
```

#### C. FFT → XY Coordinates:
```python
# Convert FFT polar to Cartesian coordinates
df_fft = convert_fft_to_xy_coordinates(df_fft)
# Creates: fft_x, fft_y from distance_m, pitch_deg
```

### 3. **Temporal Synchronization**

All three systems use different sampling rates and timestamps:
- **Sonar**: ~10 Hz from image analysis
- **DVL**: ~20 Hz from navigation system  
- **FFT**: Variable rate from signal processing

**Synchronization method**:
```python
sync_df = synchronize_fft_with_other_systems(df_sonar, df_nav, df_fft)
# Uses nearest-neighbor temporal matching within time windows
```

### 4. **Scale and Unit Normalization**

#### Distance Measurements:
- All systems normalized to **meters**
- Sonar: Direct measurement in meters
- DVL: NetDistance already in meters
- FFT: distance_m already in meters

#### Angular Measurements:
- All systems normalized to **degrees**
- Sonar: sonar_angle_deg (robot heading reference)
- DVL: NetPitch converted from radians to degrees
- FFT: pitch_deg (net-relative reference)

#### Coordinate System Alignment:
```python
# Sonar angles corrected by -180° for coordinate system alignment
# This accounts for different reference frames between systems
```

### 5. **Data Quality Filtering**

Before comparison, invalid data is filtered:
- **Sonar**: Only `detection_success == True` frames
- **DVL**: Non-NaN NetDistance and NetPitch values
- **FFT**: Valid distance_m and pitch_deg measurements

### 6. **Final Comparable Format**

After all transformations, synchronized data contains:
- **Time**: Common timestamp column
- **Distance**: All in meters, same reference frame
- **Pitch**: All in degrees, aligned coordinate systems
- **XY Position**: Cartesian coordinates for spatial comparison
- **Quality flags**: Success/validity indicators

This pipeline ensures that measurements from three fundamentally different sensor systems can be directly compared for consistency analysis.

## 8. Generate Synchronized Video with All Three Systems

Create a video overlay that shows the synchronized measurements from all three systems using the same temporal alignment approach as the comparison analysis.

In [ ]:
# SYNCHRONIZED VIDEO GENERATION WITH THREE-SYSTEM OVERLAYS
print("=== SYNCHRONIZED VIDEO GENERATION ===")

if df_sonar is not None and df_nav is not None and df_fft is not None:
    import utils.sonar_and_foto_generation as sg
    
    print("Preparing synchronized datasets for video generation...")
    
    # STEP 1: Use the same synchronization approach as the comparison analysis
    # This ensures temporal alignment between all three systems
    
    # Get synchronized dataframe from the comparison analysis
    if 'sync_df' in locals() and sync_df is not None:
        print(f"✓ Using existing synchronized data: {len(sync_df)} records")
        
        # Extract synchronized measurements for video overlay
        sonar_sync = sync_df[['timestamp', 'sonar_distance_m', 'sonar_angle_deg', 'detection_success']].copy()
        sonar_sync = sonar_sync.rename(columns={
            'sonar_distance_m': 'distance_meters',
            'sonar_angle_deg': 'angle_degrees'
        })
        
        # Add required columns for video generation compatibility
        sonar_sync['distance_pixels'] = sonar_sync['distance_meters'] / 0.01  # Approximate pixel conversion
        sonar_sync['tracking_status'] = 'SYNCHRONIZED'
        
        # Extract DVL data with proper timestamp format
        dvl_sync = df_nav.copy()
        dvl_sync['timestamp'] = pd.to_datetime(dvl_sync['timestamp'], utc=True)
        
        # Extract FFT data with synchronized timestamps
        fft_sync = df_fft.copy()
        # Use sonar reference time for FFT synchronization (same as comparison analysis)
        if 'timestamp' in df_sonar.columns:
            sonar_reference = pd.to_datetime(df_sonar['timestamp'].iloc[0], utc=True)
            
            # Convert FFT relative timestamps to absolute timestamps
            if 'time' in fft_sync.columns:
                fft_relative = pd.to_numeric(fft_sync['time'], errors='coerce')
                if fft_relative.max() < 86400:  # Relative timestamps
                    fft_sync['timestamp'] = sonar_reference + pd.to_timedelta(fft_relative, unit='s')
                else:
                    fft_sync['timestamp'] = pd.to_datetime(fft_sync['time'], utc=True)
            
            # Ensure FFT has required columns
            if 'distance' in fft_sync.columns:
                fft_sync['distance_m'] = fft_sync['distance']
            if 'pitch' in fft_sync.columns:
                fft_sync['pitch_deg'] = fft_sync['pitch']
        
        print(f"✓ Prepared synchronized datasets:")
        print(f"  Sonar: {len(sonar_sync)} records")
        print(f"  DVL: {len(dvl_sync)} records") 
        print(f"  FFT: {len(fft_sync)} records")
        
        # STEP 2: Convert timezone-aware timestamps to naive for video generation compatibility
        print("Converting timestamps for video generation compatibility...")
        
        sonar_naive = sonar_sync.copy()
        sonar_naive['timestamp'] = sonar_naive['timestamp'].dt.tz_localize(None) if sonar_naive['timestamp'].dt.tz else sonar_naive['timestamp']
        
        fft_naive = fft_sync.copy()
        fft_naive['timestamp'] = fft_naive['timestamp'].dt.tz_localize(None) if fft_naive['timestamp'].dt.tz else fft_naive['timestamp']
        
        # STEP 3: Create temporary navigation file for video generation
        from utils.sonar_config import EXPORTS_SUBDIRS
        temp_nav_file = None
        if dvl_sync is not None:
            nav_file = EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('by_bag', 'by_bag') / f"navigation_plane_approximation__{TARGET_BAG}_data.csv"
            
            if not nav_file.exists():
                nav_file.parent.mkdir(parents=True, exist_ok=True)
                # Save DVL data in expected format
                nav_export = dvl_sync.copy()
                nav_export['ts_utc'] = nav_export['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S.%f+00:00')
                nav_export.to_csv(nav_file, index=False)
                temp_nav_file = nav_file
                print(f"✓ Created temporary nav file for video generation")
        
        # STEP 4: Generate synchronized video with all three systems
        try:
            print("Generating synchronized video with three-system overlays...")
            
            video_path = sg.export_optimized_sonar_video(
                TARGET_BAG=TARGET_BAG,
                EXPORTS_FOLDER=EXPORTS_FOLDER,
                START_IDX=1,
                END_IDX=min(1200, len(sonar_naive)),
                STRIDE=1,
                AUTO_DETECT_VIDEO=True,
                INCLUDE_NET=True,
                SONAR_RESULTS=sonar_naive,       # Synchronized sonar analysis
                FFT_NET_DATA=fft_naive,          # Synchronized FFT data  
                NET_DISTANCE_TOLERANCE=0.5,     # Same tolerance as comparison
                NET_PITCH_TOLERANCE=2.0
            )
            
            print(f"\n✅ SYNCHRONIZED THREE-SYSTEM VIDEO GENERATED")
            print(f"Video path: {video_path}")
            print(f"\nColor coding (same as comparison analysis):")
            print(f"   🟡 Yellow/Orange: DVL NetDistance @ NetPitch")
            print(f"   🟣 Magenta: Sonar distance @ angle")
            print(f"   🔵 Cyan: FFT distance @ pitch")
            print(f"\nSynchronization method:")
            print(f"   - Same temporal alignment as comparison analysis")
            print(f"   - Nearest-neighbor matching within tolerance windows")
            print(f"   - All systems referenced to sonar timebase")
            
        except Exception as e:
            print(f"❌ Video generation failed: {e}")
            print(f"Error type: {type(e).__name__}")
            
        finally:
            # Clean up temporary file
            if temp_nav_file and temp_nav_file.exists():
                temp_nav_file.unlink()
                print(f"✓ Cleaned up temporary nav file")
    
    else:
        print("✗ No synchronized data available from comparison analysis")
        print("Please run the three-system comparison analysis first")

else:
    print("✗ Missing one or more datasets for video generation")
    print("Available data:")
    print(f"  Sonar: {'✓' if df_sonar is not None else '✗'}")
    print(f"  Navigation: {'✓' if df_nav is not None else '✗'}")
    print(f"  FFT: {'✓' if df_fft is not None else '✗'}")